In [27]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from client_utils import*

In [3]:
# Add an additional column, mapping to the type
df = pd.read_csv('train.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


df['binary_type'] =  df['diagnosis'].map(diagnosis_dict_binary.get)
df['type'] = df['diagnosis'].map(diagnosis_dict.get)
df.head()

,id_code,diagnosis,binary_type,type
0,000c1434d8d7,2,DR,Moderate
1,001639a390f0,4,DR,Proliferate_DR
2,0024cdab0c1e,1,DR,Mild
3,002c21358ce6,0,No_DR,No_DR
4,005b95c28852,0,No_DR,No_DR


In [4]:
# path to the data folder
img_path = 'gaussian_filtered_images/imgs'

#the path list 
image_paths = list(paths.list_images(img_path))

image_list = load(image_paths, verbose=10000)

In [5]:
df = df.drop(['diagnosis','binary_type'],axis=1)
df['id_code'] = df['id_code']+".png"

In [6]:
#binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(df['type'])
label_df = pd.DataFrame(label_list,columns=lb.classes_)
df = pd.concat([df,label_df],axis=1)

In [7]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.1, random_state=42)

In [8]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [9]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [11]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [12]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )         

C:\Users\15197\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [14]:
#randomize client data - using keys
client_names= list(clients_batched.keys())
random.shuffle(client_names)
for client in client_names:
    print(clients_batched[client])

<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 50176), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), 

In [15]:
#global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(50176, 5)
        
#global model training loop
for comm_round in range(comms_round):
            
    # global model's weights - initial weights for all local models
    global_weights = global_model.get_weights()
    
    #list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data 
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(50176, 5)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #local model weight = weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=10, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory
        K.clear_session()
        
    # the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

12/12 [==============================] - 0s 9ms/step
comm_round: 0 | global_acc: 47.411% | global_loss: 1.5259757041931152
12/12 [==============================] - 0s 9ms/step
comm_round: 1 | global_acc: 47.411% | global_loss: 1.4899598360061646
12/12 [==============================] - 0s 10ms/step
comm_round: 2 | global_acc: 47.411% | global_loss: 1.490692377090454
12/12 [==============================] - 0s 9ms/step
comm_round: 3 | global_acc: 47.411% | global_loss: 1.4905803203582764
12/12 [==============================] - 0s 8ms/step
comm_round: 4 | global_acc: 47.411% | global_loss: 1.490466594696045
12/12 [==============================] - 0s 7ms/step
comm_round: 5 | global_acc: 47.411% | global_loss: 1.4897032976150513
12/12 [==============================] - 0s 9ms/step
comm_round: 6 | global_acc: 47.411% | global_loss: 1.4901502132415771
12/12 [==============================] - 0s 8ms/step
comm_round: 7 | global_acc: 47.411% | global_loss: 1.4902567863464355
12/12 [==========

In [23]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(50176, 5) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=160, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

12/12 [==============================] - 0s 9ms/step
comm_round: 1 | global_acc: 74.387% | global_loss: 1.2148629426956177
